In [1]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


··········


In [4]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "[nick name list] james = patrick", "question": "what is james's nickname"}
).to_messages()

example_messages

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: what is james's nickname \nContext: [nick name list] james = patrick \nAnswer:", additional_kwargs={}, response_metadata={})]

In [20]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
docs = [ WebBaseLoader(url).load() for url in urls ]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

3

In [21]:
# url 들 가지고 벡터 스토어까지 생성
splits = text_splitter.split_documents(docs_list)

# 임베딩
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# 벡터 스토어
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [78]:
question = "agent memory"

In [47]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

retriever = vectorstore.as_retriever()

system = """
내 입력값과 retrieved chunk 사이의 relevance 를 측정해줘.
애매한 경우에는 no 로 하면 좋겠어. 내가 너한테 기대하는 결과값은 relevance 키만 가지고 있는 single-key json 이야.
binary score 에 'yes' 또는 'no'를 부여하여 문서가 질문과 관련이 있는지 여부를 나타내줘.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n context: {context} "),
    ]
)

retrieval_grader = prompt | llm | JsonOutputParser()
docs = retriever.invoke(question)

docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Agent System Overview#'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Agent System Overview#'),
 Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Agent System Overview#'),
 Document(metadata={'source': 'https://

In [58]:
generation = retrieval_grader.invoke({"context": docs, "question": question})
print(generation)

{'relevance': 'yes'}


In [77]:
from langchain_core.output_parsers import StrOutputParser

if generation['relevance'] == 'yes':
  system = """
  너의 답변에 할루시네이션이 있는지 스스로 평가해줘
  내가 너한테 기대하는 결과값은 ‘hallucination’ 키만 가지고 있는 single-key json 이야.
  binary score 에 'yes' 또는 'no'를 부여하여 문서가 질문과 관련이 있는지 여부를 나타내줘.
  """

  prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system),
          ("human", "question: {question}\n\n context: {context} "),
      ]
  )

  halucination_checker = prompt | llm | JsonOutputParser()

  attempts = 0
  is_halucinated = True

  while is_halucinated and attempts < 2:
    attempts += 1
    docs = retriever.invoke(question)
    res = halucination_checker.invoke({"context": docs, "question": question})
    is_halucinated = res['hallucination'] == 'yes'

if is_halucinated:
  print("result is halucinated")
else:
  prompt = hub.pull("rlm/rag-prompt")

  rag_chain = (
      {"context": retriever, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

  final_result = rag_chain.invoke(question)

  print(final_result)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Agent memory refers to the component in an autonomous agent system that allows the agent to retain and utilize past experiences or information to inform future actions. In systems powered by large language models (LLMs), this memory can enhance problem-solving capabilities and the overall effectiveness of the agent. For more details, you can refer to the source [here](https://lilianweng.github.io/posts/2023-06-23-agent/).


In [69]:
# prompt: docs

print(docs)


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Agent System Overview#'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Agent System Overview#'), Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Agent System Overview#'), Document(metadata={'source': 'https://lil

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")